# Exploring the "clean" ADNI dataset using Pandas

The "clean" ADNI dataset is the dataset for year 5 (i.e. 5 years after baseline) cleaned by Xiao (Gavin) Gao in the following way:
1. Only data for a particular "ADNI" were used.
    - Different versions of ADNI used different versions of freesurfer which, in turn, impacted how Intra Cranial Volume is calculated. It's hard to compare, therefore, data across ADNI datasets
2. Some of the images are of very low quality so those are not considered (see `ADNI_123_V4.mat`).
3. If volume of (how many?) regions increase between 2 visits, this data is also discarded since we expect attrophy to increase -- or stay constant -- within the period of 5 years.

Lets start by importing our useful libraries

In [1]:
from scipy.io import loadmat
import os
import numpy as np
from pathlib import Path
import pandas as pd

## Loading data in to dataframe

Let's load the data into a pandas dataframe.
First lets find the file.

In [5]:
def path_to_file(filename):
    '''
    Returns path for file 'filename`. 
    Assumes file to be in the relative path: '../data/'
    '''
    here_dir    = os.path.dirname(os.path.realpath('__file__'))
    par_dir = os.path.abspath(os.path.join(here_dir, os.pardir))
    dataset_dir = os.path.join(par_dir, 'data',str(filename))
    return dataset_dir

In [16]:
adni_5y = loadmat(path_to_file('vec_a2b_5y_clean.mat'))
#get the proper matrix from the .mat file, ignoring metadata
adni_5y = adni_5y['vec_a2b_5y']
#transform into a pandas dataframe


Let's take a look at what this data looks like. It is a list of dictionaries. 